In [ ]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [ ]:
pop_2010_2019 = pd.read_excel('../data/raw_data/TN/pop_est_2010-2019_tn.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

In [ ]:
pop_2020_2021 = pd.read_excel('../data/raw_data/TN/pop_est_2020-2021_tn.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

In [ ]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined['County'] = pop_combined['County'].str.strip()
pop_combined['County'] = pop_combined['County'].str.upper()
pop_combined.tail(20)

In [ ]:
metros = pd.read_excel('../data/raw_data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_tn = metros_df[metros_df['State Name']=='Tennessee']
metros_tn['County'] = metros_tn['County'].str.strip()
metros_tn['County'] = metros_tn['County'].str.upper()
metros_tn.head()

In [ ]:
file ='../data/raw_data/TN/TN_2010_2022.pdf'

In [ ]:
pdf = pdfplumber.open(file)
table = pdf.pages[43].extract_table()
text = pdf.pages[43].extract_text()
table = table [1:]
last_ele = table[-1]
for ele in last_ele:
     match = re.search("(Total\s\s\d{4})",ele)
     boolean = bool(match)
     year = re.findall("(?<=Total\s\s)\d{4}",ele)
     if boolean ==True:
        print(year,"Found!")

### Extracting data for years 2013 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [ ]:
pages = list(range(12,44))
tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
pdf = pdfplumber.open(file)
for page in pages:            
    tbl = pdf.pages[page].extract_table()
    last_ele = tbl[-1]
    df = pd.DataFrame(tbl[1::],columns=['County','fatalities','injuries','PDO','total_crashes'])
    tn_df = tn_df.append(df)    
    for ele in last_ele:
        match = re.search("(Total\s\s\d{4})",ele)
        boolean = bool(match)
        year = re.findall("(?<=Total\s\s)\d{4}",ele)
        for var in year:
             name = var         
        if boolean == True:
            name = int(name)
            tn_df = tn_df.drop(['PDO'],axis=1)
            tn_df = tn_df.iloc[:-1,:]
            tn_df['County'] = tn_df['County'].str.strip()
            tn_df['County'] = tn_df['County'].str.upper()
            df_combined_tn = pd.merge(tn_df,pop_combined[['County', name]],on="County",how='left')
            df_combined_tn_new = pd.merge(df_combined_tn,metros_tn[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
            df_combined_tn_new.rename(columns = {name:'population'}, inplace = True)
            df_combined_tn_new.insert(0,'Year',name)
            df_combined_tn_new.to_excel('../data/cleaned_data/%s_TN.xlsx' % name, engine='xlsxwriter',index=False)
            tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
        else :
            continue

### Combining all the files into a master file

In [ ]:
data_file_folder ='../data/cleaned_data/'
df = []
for file in os.listdir(data_file_folder):
        if file.endswith('_TN.xlsx'):             
             df.append(pd.read_excel(os.path.join(data_file_folder,file),sheet_name='Sheet1'))
df_tn_master = pd.concat(df, axis = 0)

In [ ]:
df_tn_master.isna().sum()

In [ ]:
df_tn_master.insert(1,'State','Tennessee') 
df_tn_master.head(10)

In [ ]:
df_tn_master.tail(10)

In [ ]:
df_tn_master.to_excel('../data/TN.xlsx', engine='xlsxwriter',index=False)

In [ ]:
df_tn_master_pop_gr = df_tn_master.groupby(["CBSA Title","Year","Metropolitan/Micropolitan Statistical Area"])["population"].sum().reset_index()
df_tn_master_pop_gr

In [ ]:
df_tn_master_pop_gr = pd.pivot(df_tn_master_pop_gr,index=["CBSA Title","Metropolitan/Micropolitan Statistical Area"],columns=["Year"])
df_tn_master_pop_gr

In [ ]:
df_tn_master_pop_analysis = df_tn_master_pop_gr.pct_change(axis='columns')
df_tn_master_pop_analysis